* [1. Problem Statement](#0)
* [2. Getting Started - Load Libraries and Dataset](#1)
    * [2.1. Load Libraries](#1.1)    
    * [2.2. Load Dataset](#1.2)

<a id='0'></a>
# 1. Problem Statement

link: https://github.com/RomelTorres/alpha_vantage

In [ ]:
https://medium.com/codex/alpha-vantage-an-introduction-to-a-highly-efficient-free-stock-api-6d17f4481bf

In [ ]:
Content:
    Stock
    Many stocks/Indices
    Crypto
    FX
    Finacial
    Real Time Data
    List of tickers available 

In [11]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import time
import requests
from io import BytesIO


In [2]:
# API Key
key = "AAUUBAYEDCI353AC"

In [27]:
ts = TimeSeries(key=key, output_format='pandas')
data = ts.get_daily(symbol='TSLA', outputsize='full')[0]
data.columns = ["open","high","low","close","volume"]
data = data.iloc[::-1]

In [4]:
data

,open,high,low,close,volume
date,,,,,
2010-06-29,19.00,25.0000,17.5400,23.89,18766300.0
2010-06-30,25.79,30.4192,23.3000,23.83,17187100.0
2010-07-01,25.00,25.9200,20.2700,21.96,8218800.0
2010-07-02,23.00,23.1000,18.7100,19.20,5139800.0
2010-07-06,20.00,20.0000,15.8300,16.11,6866900.0
...,...,...,...,...,...
2021-12-17,914.77,960.6599,909.0401,932.57,33626754.0
2021-12-20,910.70,921.6884,893.3900,899.94,18826671.0
2021-12-21,916.87,939.5000,886.1200,938.53,23839305.0


### Intraday Data

In [8]:
def get_intraday_data(symbol, interval):
    api_key = key
    api_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&apikey={api_key}'
    raw_df = requests.get(api_url).json()
    df = pd.DataFrame(raw_df[f'Time Series ({interval})']).T
    df = df.rename(columns = {'1. open': 'open', '2. high': 'high', '3. low': 'low', '4. close': 'close', '5. volume': 'volume'})
    for i in df.columns:
        df[i] = df[i].astype(float)
    df.index = pd.to_datetime(df.index)
    df = df.iloc[::-1]
    return df

tsla_intra = get_intraday_data('TSLA', '1min')
tsla_intra.tail()

,open,high,low,close,volume
2021-12-23 19:56:00,1068.45,1068.90,1068.45,1068.5,2502.0
2021-12-23 19:57:00,1068.70,1068.70,1068.70,1068.7,855.0
2021-12-23 19:58:00,1069.00,1069.50,1069.00,1069.5,3332.0
2021-12-23 19:59:00,1069.20,1069.72,1069.05,1069.5,2642.0
2021-12-23 20:00:00,1069.00,1069.30,1069.00,1069.3,4489.0


### Live Updates

In [15]:
def get_live_updates(symbol):
    api_key = key
    api_url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={api_key}'
    raw_df = requests.get(api_url).json()
    attributes = {'attributes':['symbol', 'open', 'high', 'low', 'price', 'volume', 'latest trading day', 'previous close', 'change', 'change percent']}
    attributes_df = pd.DataFrame(attributes)
    values = []
    for i in list(raw_df['Global Quote']):
        values.append(raw_df['Global Quote'][i])
    values_dict = {'values':values}
    values_df = pd.DataFrame(values).rename(columns = {0:'values'})
    frames = [attributes_df, values_df]
    df = pd.concat(frames, axis = 1, join = 'inner').set_index('attributes')
    return df

ibm_updates = get_live_updates('IBM')
ibm_updates

,values
attributes,
symbol,IBM
open,130.0000
high,130.9600
low,129.5200
price,130.6300
volume,3649044
latest trading day,2021-12-23
previous close,129.7500
change,0.8800


### Earnings Information

In [14]:
BASE_URL = r"https://www.alphavantage.co/query?"

def earnings_calendar_api(api_key, horizon, symbol=None):
    if symbol is not None:
        url = f'{BASE_URL}function=EARNINGS_CALENDAR&symbol={symbol}&horizon={horizon}&apikey={api_key}'
        response = requests.get(url)
    else:
        url = f"{BASE_URL}function=EARNINGS_CALENDAR&horizon={horizon}&apikey={api_key}"
        response = requests.get(url)

    return pd.read_csv(BytesIO(response.content))

api_key = key
horizon=["3month","6month","12month"]
earnings_calendar_api(api_key,horizon[0])

,symbol,name,reportDate,fiscalDateEnding,estimate,currency
0,A,Agilent Technologies Inc,2022-02-14,2022-01-31,1.17,USD
1,AA,Alcoa Corp,2022-01-18,2021-12-31,1.77,USD
2,AADI,Aadi Bioscience Inc,2022-03-09,2021-12-31,-1.06,USD
3,AAIC,Arlington Asset Investment Corp - Class A,2022-02-14,2021-12-31,0.06,USD
4,AAL,American Airlines Group Inc,2022-01-26,2021-12-31,-1.79,USD
...,...,...,...,...,...,...
4788,ZUO,Zuora Inc - Class A,2022-03-09,2022-01-31,-0.02,USD
4789,ZVO,Zovio Inc,2022-02-22,2021-12-31,-0.11,USD
4790,ZYME,Zymeworks Inc,2022-02-22,2021-12-31,-1.18,USD
4791,ZYNE,Zynerba Pharmaceuticals Inc,2022-03-08,2021-12-31,-0.26,USD


## Cryptocurrencies

In [54]:
url = "https://www.alphavantage.co/query"

### Historic Data

In [60]:
data = {
    "function": "DIGITAL_CURRENCY_DAILY", # WEEKLY, MONTHLY possible
    "symbol": "ETH",
    "market": 'CNY',
    "apikey": key
    }
r = requests.get(url, params=data)
data = r.json()
crypto_df = pd.DataFrame(data['Time Series (Digital Currency Daily)']).T.reset_index()
crypto_df = crypto_df.rename(columns={"index": "Date"})
crypto_df['Date'] = pd.to_datetime(crypto_df['Date'])
crypto_df


,Date,1a. open (CNY),1b. open (USD),2a. high (CNY),2b. high (USD),3a. low (CNY),3b. low (USD),4a. close (CNY),4b. close (USD),5. volume,6. market cap (USD)
0,2021-12-27,25883.31547200,4063.57000000,25990.96171200,4080.47000000,25780.12795200,4047.37000000,25954.40020800,4074.73000000,14823.90760000,14823.90760000
1,2021-12-26,26079.43545600,4094.36000000,26147.20800000,4105.00000000,25478.40000000,4000.00000000,25883.25177600,4063.56000000,189309.25500000,189309.25500000
2,2021-12-25,25773.63096000,4046.35000000,26356.83153600,4137.91000000,25597.00195200,4018.62000000,26082.42916800,4094.83000000,208126.88050000,208126.88050000
3,2021-12-24,26188.03713600,4111.41000000,26337.27686400,4134.84000000,25587.76603200,4017.17000000,25773.69465600,4046.36000000,230833.16610000,230833.16610000
4,2021-12-23,25347.88689600,3979.51000000,26433.84000000,4150.00000000,24798.31780800,3893.23000000,26188.03713600,4111.41000000,380207.81660000,380207.81660000
...,...,...,...,...,...,...,...,...,...,...,...
995,2019-04-07,1054.55097600,165.56000000,1117.41892800,175.43000000,1044.99657600,164.06000000,1109.64801600,174.21000000,465507.52497000,465507.52497000
996,2019-04-06,1049.96486400,164.84000000,1100.47579200,172.77000000,1014.04032000,159.20000000,1054.16880000,165.50000000,528168.33297000,528168.33297000
997,2019-04-05,1002.12916800,157.33000000,1069.00996800,167.83000000,993.27542400,155.94000000,1049.51899200,164.77000000,516070.34553000,516070.34553000
998,2019-04-04,1018.88121600,159.96000000,1044.61440000,164.00000000,973.27488000,152.80000000,1002.32025600,157.36000000,686080.95248000,686080.95248000


### Exchange Rates


In [98]:
data = {
    "function": "CURRENCY_EXCHANGE_RATE", # WEEKLY, MONTHLY possible
    "from_currency": "ETH",
    "to_currency": 'USD',
    "apikey": key
    }
r = requests.get(url, params=data)
data = r.json()
crypto_df = pd.DataFrame(data['Realtime Currency Exchange Rate'], index=[0]).T
crypto_df

,0
1. From_Currency Code,ETH
2. From_Currency Name,Ethereum
3. To_Currency Code,USD
4. To_Currency Name,United States Dollar
5. Exchange Rate,4095.02000000
6. Last Refreshed,2021-12-27 18:57:01
7. Time Zone,UTC
8. Bid Price,4095.29000000
9. Ask Price,4095.30000000


### Economic Indicators

In [91]:
gdp = {
    "function": "REAL_GDP",
    "interval": "annual", # quarterly
    "apikey": key
}
treasury_yield = {
    "function": "TREASURY_YIELD",
    "interval": "weekly", # daily, monthly
    "maturity": "3month", # OPTIONAL 5year, 10year, 30year
    "apikey": key
}
federal_funds_rate = {
    "function": "FEDERAL_FUNDS_RATE",
    "interval": "weekly", # daily, monthly
    "apikey": key
}
cpi = {
    "function": "CPI",
    "interval": "weekly", # daily, monthly
    "apikey": key
}
inflation = {
    "function": "INFLATION",
    "interval": "weekly", # daily, monthly
    "apikey": key
}
consumer_sentiment = {
    "function": "CONSUMER_SENTIMENT",
    "apikey": key
}
unemployment = {
    "function": "UNEMPLOYMENT",
    "apikey": key
}
r = requests.get(url, params=unemployment) # REPLACE 'params' with desired dict
data = r.json()
crypto_df = pd.DataFrame(data['data'])
crypto_df = crypto_df.set_index("date")
crypto_df

,value
date,
2021-11-01,4.2
2021-10-01,4.6
2021-09-01,4.8
2021-08-01,5.2
2021-07-01,5.4
...,...
1948-05-01,3.5
1948-04-01,3.9
1948-03-01,4.0


### Technical Indicators

In [101]:
# Lots to cover here, I've listed the most popular below
popular_ti = {
    "function": "ADX", # REPLACE: EMA, RSI, ADX, SMA
	"symbol": "IBM",
	"interval": "weekly",
	"time_period": "10",
	"series_type": "open",
    "apikey": key
}

r = requests.get(url, params=popular_ti)
data = r.json()
crypto_df = pd.DataFrame(data['Technical Analysis: ADX']).T # REPLACE to match the function
crypto_df

,ADX
2021-12-23,23.8256
2021-12-17,24.8406
2021-12-10,26.8894
2021-12-03,29.1061
2021-11-26,28.1847
...,...
2000-04-20,28.4723
2000-04-14,31.4184
2000-04-07,34.8993
2000-03-31,34.9256


## Extracting stock data (historical close price) for multiple stocks


In [ ]:
all_tickers = ["AAPL","MSFT","CSCO","AMZN","GOOG",
               "FB","BA","MMM","XOM","NKE","INTC"]
close_prices = pd.DataFrame()
api_call_count = 1
ts = TimeSeries(key=key, output_format='pandas')
start_time = time.time()
for ticker in all_tickers:
    data = ts.get_intraday(symbol=ticker,interval='1min', outputsize='compact')[0]
    api_call_count+=1
    data.columns = ["open","high","low","close","volume"]
    data = data.iloc[::-1]
    close_prices[ticker] = data["close"]
    if api_call_count==5:
        api_call_count = 1
        time.sleep(60 - ((time.time() - start_time) % 60.0))

In [ ]:
close_prices

## Extracting OHLCV data for multiple stocks


In [ ]:
all_tickers = ["AAPL","MSFT","CSCO","AMZN","GOOG",
               "FB","BA","MMM","XOM","NKE","INTC"]
ohlv_dict = {}
api_call_count = 1
ts = TimeSeries(key=open(key_path,'r').read(), output_format='pandas')
start_time = time.time()
for ticker in all_tickers:
    data = ts.get_intraday(symbol=ticker,interval='1min', outputsize='compact')[0]
    api_call_count+=1
    data.columns = ["open","high","low","close","volume"]
    data = data.iloc[::-1]
    ohlv_dict[ticker] = data
    if api_call_count==5:
        api_call_count = 1
        time.sleep(60 - ((time.time() - start_time) % 60.0))

In [ ]:
ohlv_dict['AMZN']